In [45]:
import pandas as pd

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc, classification_report
from imblearn.over_sampling import SMOTE

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, roc_auc_score, precision_recall_curve, auc
import numpy as np
import matplotlib.pyplot as plt

In [46]:
dataset = 2

In [47]:
df = pd.read_csv(f"../data/output_site_level_dataset{dataset}.csv")

if 'Unnamed: 0' in df.columns:
  df = df.drop(columns='Unnamed: 0')

In [48]:
dataset_0 = pd.read_csv("../data/output_site_level_2.csv")

if 'Unnamed: 0' in dataset_0.columns:
  dataset_0 = dataset_0.drop(columns='Unnamed: 0')

dataset_0 = dataset_0.rename(columns={
    'ID': 'transcript_id',
    'POS': 'transcript_position',
    'SEQ': '7mer'
})


X_0 = dataset_0.drop(columns=["gene_id", "transcript_id", "transcript_position", "label"])
y_0 = dataset_0['label']

scaler = StandardScaler()
X_0_scaled = scaler.fit_transform(X_0)

In [49]:
df.head()

,ID,POS,PreTime_mean,PreTime_median,PreTime_mode,PreTime_std,PreTime_min,PreTime_max,PreTime_p25,PreTime_p75,...,PostSD_p25,PostSD_p75,PostMean_mean,PostMean_median,PostMean_mode,PostMean_std,PostMean_min,PostMean_max,PostMean_p25,PostMean_p75
0,tx_id_0,0,0.007331,0.00571,0.00232,0.004967,0.00166,0.0345,0.00368,0.009450,...,1.8600,2.6000,81.607232,81.5,81.1,2.082211,74.9,91.2,80.5,82.5
1,tx_id_0,10,0.009909,0.00805,0.00232,0.006903,0.00166,0.0667,0.00498,0.013000,...,2.3000,5.1000,74.023667,73.6,73.2,2.547105,69.4,88.8,72.4,75.1
2,tx_id_0,20,0.011476,0.01020,0.01130,0.006386,0.00166,0.0578,0.00707,0.014300,...,1.9600,2.9600,88.990429,89.2,89.2,1.870263,79.6,101.0,88.1,90.1
3,tx_id_0,30,0.008315,0.00695,0.00232,0.005611,0.00166,0.0473,0.00432,0.010375,...,2.3225,4.8475,71.245575,70.7,69.9,3.020604,65.9,84.6,69.0,73.0
4,tx_id_0,40,0.008179,0.00681,0.00432,0.005166,0.00166,0.0455,0.00447,0.010300,...,2.2800,3.7725,91.278961,91.3,91.7,1.660545,81.9,99.4,90.3,92.2


In [50]:
reads_df = df.rename(columns={
    'ID': 'transcript_id',
    'POS': 'transcript_position',
    'SEQ': '7mer'
})

In [51]:
X = reads_df.drop(columns=["transcript_id", "transcript_position"])
X_scaled = scaler.transform(X)

In [52]:
model = load_model("../models/site_level_model.keras")

y_pred_prob = model.predict(X_scaled)

42/42 [==============================] - 0s 2ms/step


In [53]:
y_pred_prob

array([[0.04964567],
       [0.9838379 ],
       [0.9887484 ],
       ...,
       [0.9267664 ],
       [0.4660889 ],
       [0.67965555]], dtype=float32)

In [54]:
df_transcript = reads_df[["transcript_id", "transcript_position"]]

df_transcript['score'] = y_pred_prob

df_transcript.head()

C:\Users\nicho\AppData\Local\Temp\ipykernel_30532\3535398477.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_transcript['score'] = y_pred_prob


,transcript_id,transcript_position,score
0,tx_id_0,0,0.049646
1,tx_id_0,10,0.983838
2,tx_id_0,20,0.988748
3,tx_id_0,30,0.996517
4,tx_id_0,40,0.949584


In [55]:
df_transcript.tail()

,transcript_id,transcript_position,score
1318,tx_id_6,1840,0.397096
1319,tx_id_6,1850,0.044984
1320,tx_id_6,1860,0.926766
1321,tx_id_6,1870,0.466089
1322,tx_id_6,1880,0.679656


In [ ]:
# df_transcript.to_csv(f"../intermediate_submissions/genetherapy_dataset{dataset}_2.csv", index=False)